# Endpoints

In [ ]:
import yaml
from pynanomapper import aa
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import requests
from importlib import reload 
from pynanomapper import client_solr
from pynanomapper import client_ambit
from pynanomapper import annotation
import pandas as pd
import numpy as np
import json
import pandas as pd
import qgrid
import warnings
warnings.simplefilter("ignore")

In [ ]:
print('Select enanoMapper aggregated search service:')
style = {'description_width': 'initial'}
config,config_servers, config_security, auth_object, msg = aa.parseOpenAPI3()    
service_widget = widgets.Dropdown(
    options=config_servers['url'],
    description='Service:',
    disabled=False,
    style=style
)
if config_security is None:
    service = interactive(aa.search_service_open,url=service_widget)
else:
    print(msg)
    apikey_widget=widgets.Text(
            placeholder='',
            description=config_security,
            disabled=False,
            style=style
    )    
    service = interactive(aa.search_service_protected,url=service_widget,apikey=apikey_widget)    

display(service)

In [ ]:
service_uri=service_widget.value
if auth_object!=None:
    auth_object.setKey(apikey_widget.value)
print("Sending queries to {}".format(service_uri))



In [ ]:
query_category="ASPECT_RATIO_SHAPE_SECTION"
query_endpoint='ASPECT'
data = {'fq': 'type_s:study', 'q': 'endpointcategory_s:'+query_category, 'wt':'json', 'indent':'on', 'rows': 1000000}
results = client_solr.post(service_uri,query=data,auth=auth_object)    
response_json=results.json()

In [ ]:
from pandas.io.json import json_normalize
df=json_normalize( response_json["response"]["docs"])
df.fillna("",inplace=True)
#print(df.columns)
#df = df[df['effectendpoint_s'].str.contains(query_endpoint)]
try:
    df['loValue_d'] = pd.to_numeric(df['loValue_d'],errors='coerce')
except Exception as e:
    pass
try:
    df['upValue_d'] = pd.to_numeric(df['upValue_d'],errors='coerce')
except Exception as e:
    pass
try:
    df['err_d'] = pd.to_numeric(df['upValue_d'],errors='coerce')
except Exception as e:
    pass
try:
    a = annotation.DictionarySubstancetypes()
    df[ 'substanceType_name']=df[ 'substanceType_s'].apply(a.annotate)    
except Exception as e:
    pass
if "endpointcategory_s" in df.columns:    
    a = annotation.DictionaryEndpointCategory()
    df[ 'endpointcategory_term']=df[ 'endpointcategory_s'].apply(a.annotate)
    a = annotation.DictionaryEndpointCategoryNames()
    df[ 'endpointcategory_name']=df[ 'endpointcategory_s'].apply(a.annotate)
#df.info()

In [ ]:
cols=[]
for col in ["owner_name_s","substanceType_name","publicname_s","endpointcategory_name","guidance_s","reference_s","reference_owner_s","effectendpoint_s","effectendpoint_type_s","loQualifier_s","loValue_d","upQualifier_s","upValue_d","err_qualifier_s","err_d","unit_s","textValue_s"]:
    if col in df.columns:
        cols.append(col)        
display(df[cols].head())


In [ ]:
table = pd.pivot_table(df, values='loValue_d', columns=['effectendpoint_s','effectendpoint_type_s','reference_owner_s'], index=["publicname_s"], aggfunc=np.mean).fillna("").reset_index()
table